word sentiment classification

In [1]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]
# token dictionary. word를 character의 seqeunce로 간주함.
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
print(''.join(words))
print(set(''.join(words)))
print(list(set(''.join(words))))

idx2char = {idx: char for idx, char in enumerate(char_set)}
char2idx = {char:idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)

goodbadworseso good
{'a', 'r', 's', 'e', 'o', 'g', 'd', ' ', 'b', 'w'}
['a', 'r', 's', 'e', 'o', 'g', 'd', ' ', 'b', 'w']
['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [2]:
x_data = list(map(lambda word :[char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word: len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [4]:
max_sequence = 10
x_data = pad_sequences(sequences=x_data, maxlen=max_sequence,
                       padding='post', truncating='post')

print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [5]:
input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx)) # 대각행렬 len(char2idx) by len(char2idx)
hidden_size =10
num_classes = 2

model=Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim= output_dim,
                           mask_zero = True, # 0으로 패딩된 부분은 연산에서 제외
                           trainable = False, # embedding layer 학습시키지 않음
                           input_length=max_sequence,
                           embeddings_initializer=keras.initializers.Constant(one_hot))) #one hot vector
model.add(layers.SimpleRNN(units=hidden_size)) 
# SimpleRNN: 시퀀스의 마지막 토큰을 인풋으로 받아 처리한 결과를 리턴
model.add(layers.Dense(units=num_classes)) #many to one
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 11)            121       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10)                220       
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 363
Trainable params: 242
Non-trainable params: 121
_________________________________________________________________


In [6]:
# output shape 보면 
# embedding 쪽은 (data dimension, max sequence, input dimension)
# simple_rnn 쪽은 (?, hidden_size)
# dense 쪽은 (?, classification)

In [7]:
def loss_fn(model,x,y):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
        y_true=y, y_pred=model(x), from_logits=True))
# y가 one hot vector가 아니기 때문에 losses.sparse_categorical_crossentropy를 씀
# cross entropy

lr = 0.1
epochs = 30
batch_size = 2
opt = tf.keras.optimizers.Adam(learning_rate = lr)

tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size= batch_size)

print(tr_dataset)

<BatchDataset shapes: ((None, 10), (None,)), types: (tf.int32, tf.int32)>


In [8]:
tr_loss_hist = []
for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0

    for x_mb, y_mb in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x= x_mb, y=y_mb)
        grads = tape.gradient(target=tr_loss, sources=model.variables)
        opt.apply_gradients(grads_and_vars=zip(grads,model.variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
        tr_loss_hist.append(avg_tr_loss)

    if (epoch + 1) % 5 ==0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   5, tr_loss : 0.011
epoch :  10, tr_loss : 0.001
epoch :  15, tr_loss : 0.000
epoch :  20, tr_loss : 0.000
epoch :  25, tr_loss : 0.000
epoch :  30, tr_loss : 0.000


In [9]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)
print('acc:{:.2%}'.format(np.mean(yhat == y_data)))

acc:100.00%
